# Simplex Method

### Required imports

In [89]:
import numpy as np
import pandas as pd

### Reading the problem from csv file

required format for the .csv file

   **max/min,  x1,  x2, x3, ...**
   
   **xc11, xc12, xc13, ... <=/>=/= b1**

   **xc21, xc22, xc23, ... <=/>=/= b2**

        ...     ...     ...



In [90]:
problem = pd.read_csv('sp1.csv', header=None)

optimization_function = problem.iloc[0].dropna() # extract the optimization function
optimization_function = optimization_function.to_numpy()
# headers = optimization_function.to_frame().iloc[0]
# optimization_function  = pd.DataFrame(optimization_function.values[1:], columns=headers)

# convert the optimization function to a maximization problem
if optimization_function[0] == 'max':
    optimization_function =  optimization_function[1:].astype(float)
else:   
    optimization_function =-1 * optimization_function[1:].astype(float)


constraints=problem.iloc[[i for i in range(1,problem.shape[0])]] # extract the constraints

print("Extracted problem from the csv file is->\n")

print("\nMaximize -> " , optimization_function)
print( "\nConstarints are ->\n",constraints.to_string(index=False, header=False))


Extracted problem from the csv file is->


Maximize ->  [16. 17. 10.]

Constarints are ->
 1 1 4 <= 2000.0
2 1 1 <= 3600.0
1 2 2 <= 2400.0
1 0 0 <=   30.0


### Introducing Slack Variables

In [91]:
# constraints = constraints.to_numpy()
b = constraints.iloc[:,-1:]

constraints = constraints.iloc[:,:-2]
constraints = constraints.to_numpy().astype(float)

b=b.to_numpy()

# number of constraints is the shape of the slacks identity matrix
slack_matrix=np.identity(constraints.shape[0])

c_j = np.append(optimization_function, np.zeros(constraints.shape[0]))

print("\n\nThe problem is converted to the following standard form ->\n")
print("\nc_j ->\n",c_j)
print("\nb ->\n",b)
print("\nconstraints ->\n",constraints)
print("\nslack_matrix ->\n",slack_matrix)

# create the initial table
table = np.hstack((b,constraints, slack_matrix))
c_B = np.zeros(constraints.shape[0]).reshape(-1,1)

print("\n\nThe initial table is ->\n")
print(c_B, table)




The problem is converted to the following standard form ->


c_j ->
 [16. 17. 10.  0.  0.  0.  0.]

b ->
 [[2000.]
 [3600.]
 [2400.]
 [  30.]]

constraints ->
 [[1. 1. 4.]
 [2. 1. 1.]
 [1. 2. 2.]
 [1. 0. 0.]]

slack_matrix ->
 [[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]


The initial table is ->

[[0.]
 [0.]
 [0.]
 [0.]] [[2.0e+03 1.0e+00 1.0e+00 4.0e+00 1.0e+00 0.0e+00 0.0e+00 0.0e+00]
 [3.6e+03 2.0e+00 1.0e+00 1.0e+00 0.0e+00 1.0e+00 0.0e+00 0.0e+00]
 [2.4e+03 1.0e+00 2.0e+00 2.0e+00 0.0e+00 0.0e+00 1.0e+00 0.0e+00]
 [3.0e+01 1.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 0.0e+00 1.0e+00]]


### Simplex iterations

In [92]:
# b_order = np.arange(constraints.shape[1]+1, table.shape[1])
# print("\n\nb_order is ->\n",b_order)

for i in range(100):
    # find the initial basic feasible solution
    z_j = np.zeros(table.shape[1]-1)
    for i in range(c_B.shape[0]):
        z_j += c_B[i] * np.delete(table, 0, 1)[i]
        print(c_B[i], "*", np.delete(table, 0, 1)[i])
    print("\n\nz_j is ->\n",z_j)


    # find the initial z_j - c_j
    z_j_c_j = z_j - c_j
    print("\n\n z_j - c_j is \n",z_j_c_j)

    

    if (z_j_c_j >= 0).all():
        print("\n\nThe solution is optimal")
        b=table[:,0]
        break
    else:
        incoming_variable_index = np.argmin(z_j_c_j)
        print("\nIcoming variable's index = ", incoming_variable_index+1,"\nCounting min ratio...")
        # find the min ratio
        min_ratio = np.inf
        for i in range(table.shape[0]):
            if table[i][incoming_variable_index+1] > 0:
                ratio = table[i][0]/table[i][incoming_variable_index+1]
                print("Ratio is ->", table[i][0],"/",table[i][incoming_variable_index+1],"=",ratio)
                if ratio < min_ratio:
                    if ratio <0:
                        print("Ratio is negative")
                        continue
                    min_ratio = ratio
                    outgoing_variable_index = i
        if min_ratio == np.inf:
            print("Unbounded solution")
            break
        print("\nOutgoing variable's index = ", outgoing_variable_index)

        print("\nThe key value is ->\n",table[outgoing_variable_index][incoming_variable_index+1])

        # c_B[outgoing_variable_index] = table[outgoing_variable_index][incoming_variable_index+1]
        c_B[outgoing_variable_index] = c_j[incoming_variable_index]
        print("\nc_B is ->\n",c_B)
        # b_order[outgoing_variable_index] = incoming_variable_index+1

        # update the table
        # z_j = np.zeros(table.shape[1]-1)

        table[outgoing_variable_index] = table[outgoing_variable_index]/table[outgoing_variable_index][incoming_variable_index+1]
        for i in range(table.shape[0]):
            if i != outgoing_variable_index:
                table[i] = table[i] - table[outgoing_variable_index]*table[i][incoming_variable_index+1]
        print("\n\nThe updated table is ->\n")
        print(c_B,table)

# print(b_order)

# find the optimal solution
# maxZ = 0
# for i in range(b_order.shape[0]):
#     print("x",b_order[i],"=",b[i])
#     maxZ += optimization_function[b_order[i]-1]*b[i]
maxZ = 0
for i in range(c_B.shape[0]):
    print(c_B[i],"*",b[i])
    maxZ += c_B[i]*b[i]

print("Max Z = ",maxZ)


[0.] * [1. 1. 4. 1. 0. 0. 0.]
[0.] * [2. 1. 1. 0. 1. 0. 0.]
[0.] * [1. 2. 2. 0. 0. 1. 0.]
[0.] * [1. 0. 0. 0. 0. 0. 1.]


z_j is ->
 [0. 0. 0. 0. 0. 0. 0.]


 z_j - c_j is 
 [-16. -17. -10.   0.   0.   0.   0.]

Icoming variable's index =  2 
Counting min ratio...
Ratio is -> 2000.0 / 1.0 = 2000.0
Ratio is -> 3600.0 / 1.0 = 3600.0
Ratio is -> 2400.0 / 2.0 = 1200.0

Outgoing variable's index =  2

The key value is ->
 2.0

c_B is ->
 [[ 0.]
 [ 0.]
 [17.]
 [ 0.]]


The updated table is ->

[[ 0.]
 [ 0.]
 [17.]
 [ 0.]] [[ 8.0e+02  5.0e-01  0.0e+00  3.0e+00  1.0e+00  0.0e+00 -5.0e-01  0.0e+00]
 [ 2.4e+03  1.5e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00 -5.0e-01  0.0e+00]
 [ 1.2e+03  5.0e-01  1.0e+00  1.0e+00  0.0e+00  0.0e+00  5.0e-01  0.0e+00]
 [ 3.0e+01  1.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  0.0e+00  1.0e+00]]
[0.] * [ 0.5  0.   3.   1.   0.  -0.5  0. ]
[0.] * [ 1.5  0.   0.   0.   1.  -0.5  0. ]
[17.] * [0.5 1.  1.  0.  0.  0.5 0. ]
[0.] * [1. 0. 0. 0. 0. 0. 1.]


z_j is ->
 [ 8.5 17